In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install impyute
!pip install catboost
!pip install optuna
!pip install BayesianOptimization
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 81 kB 3.4 MB/s 
     |████████████████████████████████| 209 kB 66.6 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 146 kB 67.3 MB/s 
     |████████████████████████████████| 49 kB 2.4 MB/s 
     |████████████████████████████████| 112 kB 73.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=23ddda5cbf88ac31989997107a6b19a329f431ad1413b0deb4e0bd0562035f62
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fc

In [ ]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from impyute.imputation.cs import fast_knn
from sklearn.utils import all_estimators
from sklearn.metrics import r2_score,mean_squared_error
# from pycaret.classification import *

# Encoder

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import optuna
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from bayes_opt import BayesianOptimization

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm, tqdm_notebook
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

#matplotlib 한글깨짐 지원
import platform

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...')
rc('axes', unicode_minus=False) 

Unknown system...


### Read Data

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/머러컴피티션/2등cat/best_snd_scale_shap/snd_sum_shap_0615.csv')
test = pd.read_csv('/content/drive/MyDrive/머러컴피티션/2등cat/best_snd_scale_shap/snd_sum_shap_te_0615.csv')

In [ ]:
ID_test = pd.DataFrame({'custid': pd.read_csv('/content/drive/MyDrive/머러컴피티션/X_test.csv', encoding='cp949').custid.unique()})
target = pd.read_csv('/content/drive/MyDrive/머러컴피티션/y_train.csv', encoding = "cp949").group

In [ ]:
train.to_csv('snd_shap.0615.csv')
test.to_csv('snd_shap_te.0615.csv')

In [ ]:
target

0        M30
1        M30
2        F20
3        F20
4        M40
        ... 
21582    M50
21583    F20
21584    M30
21585    F20
21586    F20
Name: group, Length: 21587, dtype: object

In [ ]:
ID_test

,custid
0,30001
1,30002
2,30003
3,30005
4,30007
...,...
14375,49988
14376,49990
14377,49992
14378,49993


## Build Models

In [ ]:
train = train.iloc[:,1:]
test = test.iloc[:,1:]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state = 0)

In [ ]:
X_val

,012.1,012,str_num,buyer_nm_2,143_y,0602,pc_nm_1,mean_dis,96.2,23.1,...,282_x,76_x,25.2,비비안스타킹,레지나,16_x,262_y,총쇼핑시간,part_pc_size_abbd_vegetable,17.3
701,0.263992,0.649176,0.693147,0.001871,-0.010574,-0.715735,-0.010382,0.050000,0.023738,-0.258206,...,-0.024522,0.050486,0.309503,-0.173885,-0.052019,0.038843,-0.014670,-0.831352,0.0,-0.147799
6929,0.257621,-0.601269,1.386294,-0.058664,-0.005036,0.077960,-0.078765,0.044915,-0.106950,-0.301443,...,0.021443,0.022890,0.043440,-0.173885,-0.052019,0.037580,-0.010887,1.160670,0.0,0.007926
10644,0.258437,-0.434320,1.386294,-0.079321,0.001591,-0.545657,-0.028889,0.054545,-0.164534,-0.240580,...,-0.011383,0.050922,0.170375,-0.173885,-0.052019,0.120279,0.007497,-0.496870,0.0,-0.086373
11210,0.257809,-0.589416,0.693147,-0.151650,-0.003460,-0.318888,-0.049844,0.033339,-0.042639,-0.274513,...,0.052629,0.013547,0.187767,-0.173885,-0.052019,0.089084,-0.033081,-0.535294,0.0,0.035296
12880,0.267983,1.468111,1.098612,0.053972,-0.002378,-0.318888,-0.044109,0.030769,-0.109635,-0.188123,...,0.011464,0.016134,0.056285,-0.173885,-0.052019,0.000642,0.012750,-0.507043,0.0,-0.042686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18820,0.259984,-0.135175,1.098612,0.066910,-0.047881,-0.659042,-0.045678,0.050033,-0.069007,-0.193947,...,-0.005490,0.013724,0.058594,-0.173885,-0.052019,0.044690,-0.038959,-0.685785,0.0,0.090913
5186,0.258074,-0.506331,0.693147,-0.109033,-0.005128,-0.488965,-0.001467,0.087500,-0.175884,-0.253893,...,-0.056743,0.044876,0.025533,-0.173885,-0.052019,0.119609,0.027503,-0.637673,0.0,-0.136076
1465,0.258999,-0.283670,1.098612,-0.022651,-0.050955,-0.772427,-0.028712,0.033333,-0.067770,-0.257261,...,-0.019515,0.008172,0.072776,-0.173885,-0.052019,0.091870,0.007430,-0.781879,0.0,-0.126972
14834,0.258714,-0.382615,0.693147,0.029199,-0.055505,-0.772427,0.012990,0.050000,-0.305641,-0.274660,...,-0.070115,0.008007,0.051026,-0.173885,-0.052019,0.088692,0.081355,-0.806599,0.0,-0.029089


In [ ]:
import optuna

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
import sklearn

In [ ]:
bayesian_params = {
    'max_depth':(8, 16),
    'num_leaves':(24, 64),
    'min_child_samples':(10, 200),
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1),
    'colsample_bytree':(0.5, 1),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha':(0.01, 50)
}

In [ ]:
def lgb_rmse_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree, max_bin, reg_lambda, reg_alpha):
    
    params = {
        "n_estimators":2000, 
        "learning_rate":0.02,
        'max_depth':int(round(max_depth)),
        'num_leaves':int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample':max(min(subsample, 1), 0),
        'colsample_bytree':max(min(colsample_bytree, 1), 0),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0),
        'application': 'multiclass',
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_classes' : 8,
        'n_jobs' : -1
    }
    
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)],eval_metric= 'multi_logloss', verbose= 100, 
                early_stopping_rounds= 100)
    valid_pred = lgb_model.predict_proba(X_val)
    logloss = sklearn.metrics.log_loss(y_val, valid_pred)
    
    return logloss

In [ ]:
lgbBO = BayesianOptimization(f=lgb_rmse_eval, pbounds=bayesian_params, random_state=1000)
lgbBO.maximize(init_points=5, n_iter=10)

In [ ]:
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmin(np.array(target_list)))

[1.4709542969028622, 1.4794355567244564, 1.4754881616670559, 1.4702997427278173, 1.4806821142288804, 1.4692340839543634, 1.470696723895817, 1.4766256942034708, 1.471960194443625, 1.4884037696077976, 1.4684193341335265, 1.4725809063738162, 1.4762865117709376, 1.4857606940023553, 1.4846991892711152]
maximum target index: 10


In [ ]:
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmin(np.array(target_list))]
print(max_dict)

{'target': 1.4684193341335265, 'params': {'colsample_bytree': 0.5141043094390911, 'max_bin': 254.3407418874025, 'max_depth': 15.332770159742921, 'min_child_samples': 103.21393354412879, 'min_child_weight': 46.90043392010393, 'num_leaves': 32.33623155988221, 'reg_alpha': 12.21846097676822, 'reg_lambda': 9.777330191074164, 'subsample': 0.6988643735876281}}


### Kfold

In [ ]:
X_new = train
X_te_new = test

In [ ]:
targets = pd.concat([target,target])

In [ ]:
'target': 1.4684193341335265, 'params': {'colsample_bytree': 0.5141043094390911, 'max_bin': 254.3407418874025, 'max_depth': 15.332770159742921, 'min_child_samples': 103.21393354412879, 'min_child_weight': 46.90043392010393, 'num_leaves': 32.33623155988221, 'reg_alpha': 12.21846097676822, 'reg_lambda': 9.777330191074164, 'subsample': 0.6988643735876281}}

In [ ]:
from sklearn.model_selection import KFold
ftr = X_new
target = target
test_proba = []

def train_apps_all_with_oof(ftr, ttmp_arget, nfolds=5):
    ftr = ftr
    tmp_target = target

    # nfolds 개의 cross validatin fold set을 가지는 KFold 생성 
    folds = KFold(n_splits = nfolds, shuffle=True, random_state=0)
    
    # Out of Folds로 학습된 모델의 validation set을 예측하여 결과 확률을 담을 array 생성.
    # validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
    oof_preds = np.zeros((ftr.shape[0],))  
    
    # Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
    test_preds = np.zeros(((X_te_new.shape[0],)))
    
    # n_estimators를 4000까지 확대. 
    clf = LGBMClassifier(
                nthread=4,
                n_estimators=4000,
                learning_rate=0.01,
                max_depth=16,
                num_leaves=32,
                colsample_bytree=0.5141,
                subsample=0.698,
                max_bin=255,
                reg_alpha=12.218,
                reg_lambda=9.777,
                min_child_weight=47,
                min_child_samples=103,
                silent=-1,
                verbose=-1,
                )

    # nfolds 번 cross validation Iteration 반복하면서 OOF 방식으로 학습 및 테스트 데이터 예측
    for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(ftr)):
        print('##### iteration ', fold_idx, ' 시작')
        # 학습용 데이터 세트의 인덱스와 검증용 데이터 세트의 인덱스 추출하여 이를 기반으로 학습/검증 데이터 추출
        train_x = ftr.iloc[train_idx, :]
        train_y = target.iloc[train_idx]
        valid_x = ftr.iloc[valid_idx, :]
        valid_y = target.iloc[valid_idx]
        
        # 추출된 학습/검증 데이터 세트로 모델 학습. early_stopping은 200으로 증가. 
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'multi_logloss', verbose= 200, 
                early_stopping_rounds= 200)
        # 검증 데이터 세트로 예측된 확률 저장. 사용되지는 않음. 
        #oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)       
        # 학습된 모델로 테스트 데이터 세트에 예측 확률 계산. 
        # nfolds 번 반복 실행하므로 평균 확률을 구하기 위해 개별 수행시 마다 수행 횟수로 나눈 확률을 추후에 더해서 최종 평균 확률 계산. 
        test_proba.append( clf.predict_proba(test, num_iteration=clf.best_iteration_)/folds.n_splits)
        
        
    return clf, test_proba

In [ ]:
# 튜닝 후
clf, test_proba = train_apps_all_with_oof(ftr, target, nfolds=5)

##### iteration  0  시작
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_logloss: 1.46801	training's multi_logloss: 1.46801	valid_1's multi_logloss: 1.57282	valid_1's multi_logloss: 1.57282
[400]	training's multi_logloss: 1.31304	training's multi_logloss: 1.31304	valid_1's multi_logloss: 1.50739	valid_1's multi_logloss: 1.50739
[600]	training's multi_logloss: 1.2069	training's multi_logloss: 1.2069	valid_1's multi_logloss: 1.48406	valid_1's multi_logloss: 1.48406
[800]	training's multi_logloss: 1.12082	training's multi_logloss: 1.12082	valid_1's multi_logloss: 1.47295	valid_1's multi_logloss: 1.47295
[1000]	training's multi_logloss: 1.04573	training's multi_logloss: 1.04573	valid_1's multi_logloss: 1.46808	valid_1's multi_logloss: 1.46808
[1200]	training's multi_logloss: 0.979325	training's multi_logloss: 0.979325	valid_1's multi_logloss: 1.46541	valid_1's multi_logloss: 1.46541
[1400]	training's multi_logloss: 0.919533	training's multi_logloss: 0.91

In [ ]:
# 기존튜닝
clf, test_proba = train_apps_all_with_oof(ftr, target, nfolds=5)

##### iteration  0  시작
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_logloss: 1.4863	training's multi_logloss: 1.4863	valid_1's multi_logloss: 1.58301	valid_1's multi_logloss: 1.58301
[400]	training's multi_logloss: 1.3413	training's multi_logloss: 1.3413	valid_1's multi_logloss: 1.51599	valid_1's multi_logloss: 1.51599
[600]	training's multi_logloss: 1.24487	training's multi_logloss: 1.24487	valid_1's multi_logloss: 1.49088	valid_1's multi_logloss: 1.49088
[800]	training's multi_logloss: 1.16924	training's multi_logloss: 1.16924	valid_1's multi_logloss: 1.47883	valid_1's multi_logloss: 1.47883
[1000]	training's multi_logloss: 1.10567	training's multi_logloss: 1.10567	valid_1's multi_logloss: 1.47283	valid_1's multi_logloss: 1.47283
[1200]	training's multi_logloss: 1.05084	training's multi_logloss: 1.05084	valid_1's multi_logloss: 1.46936	valid_1's multi_logloss: 1.46936
[1400]	training's multi_logloss: 1.00357	training's multi_logloss: 1.00357	v

In [ ]:
test_mean_proba = (test_proba[0] + test_proba[1] + test_proba[2] + test_proba[3] + test_proba[4])

In [ ]:
pd.DataFrame(test_mean_proba)

,0,1,2,3,4,5,6,7
0,0.139610,0.074587,0.030491,0.072278,0.187954,0.302179,0.018625,0.174276
1,0.011869,0.062253,0.406078,0.017064,0.002916,0.018097,0.456634,0.025088
2,0.650935,0.275990,0.023402,0.008638,0.013567,0.015755,0.005808,0.005906
3,0.547519,0.100622,0.036042,0.053024,0.040552,0.179070,0.017899,0.025272
4,0.910071,0.043364,0.008755,0.004508,0.019658,0.006680,0.003627,0.003336
...,...,...,...,...,...,...,...,...
14375,0.114101,0.465158,0.067049,0.030743,0.021225,0.193407,0.096286,0.012033
14376,0.409944,0.248590,0.109185,0.051155,0.033093,0.078959,0.031419,0.037655
14377,0.333580,0.203235,0.109264,0.078947,0.069928,0.107735,0.039104,0.058207
14378,0.099221,0.475003,0.094765,0.013873,0.020675,0.235897,0.052585,0.007980


In [ ]:
len(test_proba[0])

6477

## Deployment

In [ ]:
ID_test

,custid
0,30001
1,30002
2,30003
3,30005
4,30007
...,...
14375,49988
14376,49990
14377,49992
14378,49993


In [ ]:
t = pd.Timestamp.now()
fname = f"lgbm_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
pred = pd.DataFrame(test_mean_proba)
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
submissions = pd.concat([pd.Series(ID_test.custid, name = 'ID'), pred], axis = 1)
submissions.to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

'lgbm_submission_06150616.csv' is ready to submit.
